In [20]:
from app.core.model_builder import ModelBuilder
from sixgill.pipesim import Model
from sixgill.definitions import ModelComponents
from app.project import get_string_values_from_class
from sixgill.core.model_context import ContextError
from sixgill.core.mapping import ParameterError

In [ ]:
model = Model.open("trail.pips")

In [3]:
components = get_string_values_from_class(ModelComponents)

In [ ]:
len(components)

In [5]:
values = {}
for component in components:
    try:
        v = model.get_values(
            component=component,
        )
        values[component] = v
    except ContextError:
        pass

In [6]:
# create df from values of each key and write it in asingle excel file with each sheet as a component

import pandas as pd

with pd.ExcelWriter("trail.xlsx") as writer:
    for key, value in values.items():
        df = pd.DataFrame(value)
        # transpose the df
        df = df.T
        df.to_excel(writer, sheet_name=key)

In [ ]:
df = pd.read_excel("trail.xlsx", sheet_name=None, index_col=0)
df["Source"]

In [ ]:
for key, value in df.items():
    print(key)
    try:
        model.set_values(dict=value.to_dict(orient="index"))
    except ParameterError:
        print(f"Error setting values for {key}")
    except Exception as e:
        print(f"Error setting values for {key}: {e}")

In [ ]:
x = model.get_values(component=ModelComponents.FLOWCORRELATION)

In [ ]:
df["Source"]

In [29]:
model.set_values(dict=df["Source"].to_dict(orient="index"))

In [ ]:
def export_values(excel_file, components=None):
    model = Model.open("trail.pips")

    if components is None:
        components = get_string_values_from_class(ModelComponents)

    values = {}
    for component in components:
        try:
            v = model.get_values(
                component=component,
            )
            values[component] = v
        except ContextError:
            pass

    with pd.ExcelWriter(excel_file) as writer:
        for key, value in values.items():
            df = pd.DataFrame(value)
            df = df.T
            df.to_excel(writer, sheet_name=key)


def import_values(excel_file):
    df = pd.read_excel(excel_file, sheet_name=None, index_col=0)
    model = Model.open("trail.pips")

    for key, value in df.items():
        try:
            components = get_string_values_from_class(ModelComponents)
            if key in components:
                model.set_values(dict=value.to_dict(orient="index"))
        except ParameterError:
            print(f"Error setting values for {key}")
        except Exception as e:
            print(f"Error setting values for {key}: {e}")